In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXimJyDjm9m7k8 FJoLeEVAbOl0kiGIZTWPKAtav6nWikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
analogical_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Analogical Prompting based on user questions.
    
    Analogical Prompting is a technique within Chain-of-Thought (CoT) that automatically generates exemplars with reasoning paths by drawing analogies between the current problem and similar problems. 
    
    This technique significantly improves performance on mathematical reasoning and code generation tasks by leveraging the power of analogical thinking and transferring problem-solving strategies from familiar domains to new ones.
    
    The Analogical Prompting process includes the following steps:
    1. Identify a similar problem: Find or create an analogous problem that shares structural similarities with the current problem.
    2. Solve the analogous problem: Work through the analogous problem step by step, showing the complete reasoning path.
    3. Transfer the solution strategy: Apply the same reasoning pattern from the analogous problem to the current problem.
    4. Answer the original question: Provide a final answer based on the transferred reasoning.
    
    Please structure your response in the following format:
    
    Step 1: [Analogous problem identification]
    Step 2: [Solution of the analogous problem]
    Step 3: [Transfer of solution strategy]
    Step 4: [Final Answer]
    
    Example:
    Question: "A warehouse has 25 rows of shelves with 12 boxes per shelf. If each box contains 8 items, how many total items are in the warehouse?"
    
    Step 1: Analogous problem identification:
    This problem involves multiplication across multiple levels of organization. Let me consider an analogous problem:
    "A school has 15 classrooms with 20 desks per classroom. If each desk has 2 drawers, how many total drawers are there in the school?"
    
    Step 2: Solution of the analogous problem:
    To solve this analogous problem:
    1. First, I need to find the total number of desks: 15 classrooms × 20 desks per classroom = 300 desks
    2. Then, I need to find the total number of drawers: 300 desks × 2 drawers per desk = 600 drawers
    So the school has a total of 600 drawers.
    
    Step 3: Transfer of solution strategy:
    Now I can apply the same strategy to our original problem about the warehouse:
    1. First, I need to find the total number of boxes: 25 rows × 12 boxes per shelf = 300 boxes
    2. Then, I need to find the total number of items: 300 boxes × 8 items per box = 2,400 items
    
    Step 4: The warehouse contains a total of 2,400 items.
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: Analogous problem identification:
    [Identify or create a problem that has a similar structure or solution pattern to the current question. The analogous problem should be simpler or more familiar, but maintain the key structural elements of the original problem.]
    
    Step 2: Solution of the analogous problem:
    [Work through the analogous problem step by step, showing the complete reasoning path:
    1. [First step in solving the analogous problem]
    2. [Second step in solving the analogous problem]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 3: Transfer of solution strategy:
    [Apply the reasoning pattern from the analogous problem to the original question:
    1. [First step in solving the original problem, based on the analogy]
    2. [Second step in solving the original problem, based on the analogy]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 4: [Provide a clear, concise answer to the original question, drawing directly from the transferred solution strategy]
    """)
])

In [9]:
# Fonction pour générer le prompt ThoT (node)
def generate_thot_node(state):
    question = state['messages'][-1].content  # Récupère la dernière question
    prompt_value = analogical_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    return {"messages": [response]}  # Ajoute la réponse comme un message

# Définition de l'état
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Création du graphe
graph_builder = StateGraph(State)
graph_builder.add_node("generate_thot", generate_thot_node)

# Configuration du graphe
graph_builder.set_entry_point("generate_thot")
graph_builder.add_edge("generate_thot", END)
graph = graph_builder.compile()

# Exemple d'utilisation
inputs = {"messages": [HumanMessage(content="How should I architect a distributed system for processing real-time financial transactions that needs to handle 10,000 transactions per second, maintain ACID compliance, and have a recovery time of less than 5 minutes in case of failure?")]}

# Affiche la question posée au début
print("Question posée:", inputs["messages"][0].content)
print("\nRéponse:")

for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_thot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)

Question posée: How should I architect a distributed system for processing real-time financial transactions that needs to handle 10,000 transactions per second, maintain ACID compliance, and have a recovery time of less than 5 minutes in case of failure?

Réponse:
Step 1: Analogous problem identification:
To find a similar problem, let's consider a scenario involving a high-performance system that requires reliability and quick recovery. An analogous problem could be: "How should I architect a distributed system for processing online orders for an e-commerce platform that needs to handle 5,000 orders per minute, maintain data consistency, and have a recovery time of less than 3 minutes in case of failure?"

Step 2: Solution of the analogous problem:
To solve this analogous problem:
1. First, I would consider using a microservices architecture to handle different aspects of the order processing, such as inventory management, payment processing, and order fulfillment. This allows for sca